Наша доставка еды решила запустить новое фьюжн-меню, которое сочетает ингредиенты и кулинарные традиции разных стран, создавая необычные и оригинальные блюда. Блюда фьюжн-кухни выглядят ярко и необычно, и наши клиенты обязательно оценят их уникальный вкус и оригинальную подачу.

Для части нашей аудитории, которая придерживается здорового образа жизни и следит за питанием, важно знать количество калорий, белков, жиров и углеводов в блюде. Наши блюда могут быть сложными, и на этапе приготовления бывает трудно посчитать все макронутриенты. Мы решили помочь им с помощью сервиса на основе искусственного интеллекта, который по фотографии блюда позволяет быстро узнать его пищевую ценность. Это облегчит выбор еды и сделает его более осознанным для тех, кто считает калории и макронутриенты.

Ваша задача — разработать прототип сервиса с использованием LLM (больших языковых моделей) для анализа пищевой ценности еды на фотографии. Вы хотите попробовать использовать LLM для базового и быстрого решения, чтобы далее определить, в каком направлении двигаться: обучать свою модель для точного распознавания состава продуктов или интегрировать существующие коммерческие сервисы, чтобы обеспечить наилучшее сочетание качества и скорости распознавания. Прототип поможет оценить, насколько LLM справляется с задачей, и принять решение о дальнейших шагах в развитии сервиса.

Таким образом, наш сервис будет работать следующим образом:

Пользователь загружает фото через интерфейс, созданный с помощью Streamlit
Streamlit отправляет фото на сервер, используя FastAPI сервис
FastAPI сервис отправляет запрос, содержащий изображение в LLM-модель (ассистенту) для анализа
Ассистент анализирует фото и возвращает результат анализа обратно в FastAPI сервис
FastAPI сервис отправляет полученные данные на фронтенд, где пользователь видит информацию о пищевой ценности блюда на экране.
Такой прототип позволит быстро протестировать функционал, продемонстрировать его пользователям и собрать обратную связь для дальнейшего улучшения.

Отлично, на следующем мы займемся реализацией серверной части. Мы создадим FastAPI-сервис с эндпойнтом, который будет принимать запрос с изображением в формате Base64, затем передавать его в LLM-ассистент для анализа содержимого изображения. Эндпоинт — это точка входа в API, которая обрабатывает запросы и отправляет ответы.

FastAPI
FastAPI — это современный веб-фреймворк на Python, который позволяет быстро создавать веб-приложения и API. Он очень удобен для разработчиков, так как делает работу проще и эффективнее благодаря понятному синтаксису и встроенным инструментам.

Для решения задачи мы используем ChatGPT-like модель LLaVA, предназначенную для общего понимания изображений и текста, с заявленными возможностями аналогичными мультимодальной версии GPT-4. Запуск модели будет осуществляться с помощью сервиса replicate.com.

Replicate.com — это платформа, позволяющая разработчикам легко запускать, тестировать и использовать модели машинного обучения в облаке. Одним из ключевых преимуществ использования Replicate является простота интеграции: вместо того чтобы заботиться о сложных настройках инфраструктуры, можно быстро подключить любую доступную модель через удобный API.

In [4]:
import requests
import base64

def image_url_to_base64(image_url: str) -> str:
    try:
        response = requests.get(image_url)
        response.raise_for_status()  # Проверка успешности запроса
        image_bytes = response.content
        return base64.b64encode(image_bytes).decode("utf-8")
    except requests.exceptions.RequestException as e:
        print(f"Ошибка при получении изображения: {e}")
        return ""

# Получите base64 строку
image_base64 = image_url_to_base64("https://omnomnom.dp.ua/image/cache/catalog/pizza_new/new4/pepperoni-500x500.jpg")
print("img:", image_base64)


img: ...


Prompting
На этом шаге вы напишите промпт для LLM, который на основе загруженной фотографии позволит получить ориентировочную пищевую ценность сфотографированной еды.

Промпт (prompt) — это запрос или инструкция, которую мы передаём LLM модели, чтобы она выполнила определённую задачу (например, дала текстовый ответ).

Промптинг (prompting) — это процесс создания таких запросов, которые помогут получить наиболее релевантные и точные ответы от модели. В нашем проекте хорошо сформулированный промпт позволит получить от модели примерную пищевую ценность предложенного блюда.

Промпт может включать следующие элементы:

Инструкция — описание конкретной задачи, указание стиля ответа или другие указания для модели
Контекст — дополнительная информация, которая поможет модели дать более точный результат
Входные данные — данные или вопрос, на основе которых модель должна сделать предсказание или дать ответ
Формат вывода — указание на формат или тип ожидаемых результатов
Пример промпта:

Классифицируй комментарий как нейтральный, негативный или позитивный.

Текст: Мне понравился это фильм, спасибо за рекомендацию.

Настроение комментария:


Ответ:

Позитивное

Техники промптинга
Промпт-инжиниринг помогает создавать и оптимизировать запросы для получения более качественных результатов при работе с большими языковыми моделями. Рассмотрим некоторые техники промпт-инжиниринга, которые могут помочь решить более сложные задачи, повышая точность и эффективность работы LLM.

Zero-Shot prompt:

Тип запроса к модели, в котором модель решает задачу без примеров, полагаясь только на инструкции и своё обучение. Он работает, потому что современные языковые модели обучены на огромных массивах данных, сопоставимых с размерами всего Интернета, что позволяет им развить общую "насмотренность" и эффективно понимать широкий спектр задач.
Определи, является ли это электронное письмо спамом или нет:
'Поздравляем! Вы выиграли миллион долларов! Нажмите здесь, чтобы получить приз.'

Few-Shot prompt

Тип запроса к модели, в котором даются несколько примеров решения задачи перед тем, как предложить модель решить новую, похожую задачу. Он показывает отличные результаты, потому что примеры помогают модели лучше понять контекст и ожидаемый результат, опираясь на шаблоны в предоставленных данных.

Пример 1: Текст: 'Я очень рад этой новости.' Классификация: позитивное.
Пример 2: Текст: 'Это худший день в моей жизни.' Классификация: негативное.
Текст: 'Мне всё равно, что произойдёт дальше.' Классификация:

Формат ответа:

Можно включить в промпт инструкцию, чтобы модель возвращала ответы в определённом формате, например, в формате JSON, что облегчает их дальнейшую обработку. Если модель не всегда соблюдает ожидаемый формат, также можно использовать технику few-shot, добавив 1-2 примера правильного ответа, чтобы показать модели ожидаемый результат.

Верни ответ в формате JSON с полями: calories, proteins, fats, carbohydrates
Пример ожидаемого ответа: {"calories": 100, "proteins": 10, "fats": 20, "carbohydrates": 30}
Возвращай только JSON, без лишнего текста.

Задание
На странице модели перейдите в режим Playground
Напишите промпт, чтобы для изображений, содержащих еду, в качестве ответа модель LLaVA возвращала JSON-объект с полями calories, proteins, fats, carbohydrates
Например: {"calories": 100, "proteins": 10, "fats": 20, "carbohydrates": 30}
Для изображений без еды - ответом должен быть пустой JSON-объект: {}
Добейтесь того, чтобы модель возвращала только JSON без лишнего текста
Сохраните полученный промпт и сдайте его в качестве ответа

In [5]:
def removeDuplicates(nums) -> int:
    print(len(nums))
    print(len(set(nums)))
    return len(set(nums))

In [6]:
print(removeDuplicates([1,1,2]))

3
2
2


In [21]:
def removeElement(nums, val: int) -> int:
        f, n = 0,0
        nums.sort()
        for i in range(len(nums)):
            if nums[i] == val:
                f_p = i
            if nums[-i] == val:
                n_p = -i
        nums[:] = nums[f_p:n_p] 
        return nums, len(nums)
        

In [22]:
print(removeElement([3,2,2,3],2))

([], 0)


In [36]:
def removeElement(nums, val: int) -> int:
        nums = [x for x in nums if x != val]
        return nums, len(nums)

In [37]:
print(removeElement([3,2,2,3],3))

([2, 2], 2)


In [38]:
def removeElement(nums, val: int) -> int:
    k = 0  # Initialize a pointer for the new array's length
    for i in range(len(nums)):
        if nums[i] != val:
            nums[k] = nums[i]
            k += 1
    return k


In [39]:
print(removeElement([3,2,2,3],3))

2


In [40]:
a = [1,2,3]
b = [1,2,3]

a.append([6])
print(a)

[1, 2, 3, [6]]


In [41]:
# Исходный словарь
data = {'banana': 3, 'apple': 4, 'cherry': 2, 'date': 1}

# Сортировка словаря по ключам
sorted_data = dict(sorted(data.items()))

print(sorted_data)


{'apple': 4, 'banana': 3, 'cherry': 2, 'date': 1}


In [42]:
import pandas as pd

# Создание DataFrame
data = {'temp': [22, 21, 23], 'humidity': [60, 65, 70]}
df = pd.DataFrame(data, index=[1, 2, 4])

# Прямой доступ по индексу
result = df.loc[4]  # Получаем строку с индексом 4

# Обращение к столбцу 'temp' для этой строки
temp_value = df.loc[4]['temp']


In [43]:
result

temp        23
humidity    70
Name: 4, dtype: int64

In [44]:
temp_value

np.int64(23)